In [1]:
from scripts.StreamProcessing import Stream_Data # found in the scripts folder inside StreamProcessing.py. This class uses PySpark

PySpark found
Does Data Folder exist? False


https://berkeleyearth.org/data/

### A note on block 

These might take a while as PySpark attempts to download some additional files

In [2]:
# Initialize the Stream_Data Class using the topic name/s established in the kafka-config/kafka-config.ipynb and add the host of your kafka instance in the host parameter
emissions_config = Stream_Data(topics='ghg_data', host='localhost:9092')
emissions = emissions_config.getData()
#temperatures

/media/camagakhan/DATA/Repositories/BigDataProcessing/Assignment/BigDataProcessingClimateChange/notebooks/scripts/sqlite-jdbc-3.34.0.jar


23/05/10 23:05:16 WARN Utils: Your hostname, camagakhan-Inspiron-7577 resolves to a loopback address: 127.0.1.1; using 192.168.6.33 instead (on interface wlp60s0)
23/05/10 23:05:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/camagakhan/spark-3.4.0-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/camagakhan/.ivy2/cache
The jars for the packages stored in: /home/camagakhan/.ivy2/jars
org.apache.spark#spark-streaming-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-9adf064e-b73e-4d43-8a47-52a9f3b874a9;1.0
	confs: [default]
	found org.apache.spark#spark-streaming-kafka-0-10_2.12;3.2.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.2.0 in central
	found org.apache.kafka#kafka-clients;2.8.0 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.1 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found commons-logging#com

In [3]:
#temperatures.printSchema()

In [4]:
# base_df = temperatures.selectExpr('key', 'value', 'topic', 'timestamp')
# base_df.show()

In [ ]:
# ghg_config = Stream_Data(topics='ghg_data', host='localhost:9092')
# ghg_data = ghg_config.getData()
# ghg_data

23/05/10 23:05:25 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
23/05/10 23:05:25 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
23/05/10 23:05:25 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
23/05/10 23:05:25 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
23/05/10 23:05:25 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
23/05/10 23:05:25 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
23/05/10 23:05:25 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
23/05/10 23:05:25 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config.
23/05/10 23:05:25 WARN AdminClientConfig: The co

-------------------------------------------
Batch: 0
-------------------------------------------


+------+---------+---+----------------+-----+--------------------+----+----+----------+--------------------+-------------+---------+-------------------+---------------+----------+---------+-----+
|﻿"COU"|  Country|POL|       Pollutant|  VAR|            Variable| YEA|Year|  UnitCode|                Unit|PowerCodeCode|PowerCode|ReferencePeriodCode|ReferencePeriod|     Value|FlagCodes|Flags|
+------+---------+---+----------------+-----+--------------------+----+----+----------+--------------------+-------------+---------+-------------------+---------------+----------+---------+-----+
|   AUS|Australia|GHG|Greenhouse gases|TOTAL|Total  emissions ...|1990|1990|T_CO2_EQVT|Tonnes of CO2 equ...|            3|Thousands|               null|           null|425624.307|         |     |
|   AUS|Australia|GHG|Greenhouse gases|TOTAL|Total  emissions ...|1991|1991|T_CO2_EQVT|Tonnes of CO2 equ...|            3|Thousands|               null|           null|425686.445|         |     |
|   AUS|Australia|GH